# Resolution configuration

## import tools

In [ ]:
from matplotlib import pyplot as plt
import numpy as np, os, glob
import histogram.hdf as hh, histogram as H

In [ ]:
import sys
# sys.path.insert(0, os.path.expanduser('~/dv/mcvine/dgsres/'))
import dgsres
dgsres

In [ ]:
%matplotlib inline

define and enter working dir

In [ ]:
workdir = os.path.expanduser('~/simulations/CNCS/YbCl3')
!mkdir -p {workdir}
%cd {workdir}

## Beam
* use the beam already calculated in beam.ipynb. change the path below.
* beam/out/neutrons should exist

In [ ]:
!ln -s "/path/to/beam" beam

In [ ]:
ls beam/

In [ ]:
beam = os.path.abspath("./beam/")

In [ ]:
!mcvine neutronstorage count {beam}/out/neutrons

## Sample

Please modify the following cell to match your sample specification such as chemical formula, lattice parameters, and orientation (by u,v vectors), and shape. Please make sure keep the "excitations" section intact

In [ ]:
6.328*np.cos(110.55*np.pi/180.), 6.328*np.sin(110.55*np.pi/180.)

In [ ]:
%%file sample.yaml
name: YbCl3
chemical_formula: YbCl3
lattice: 
 constants: 6.734, 11.613, 6.328, 90, 110.55, 90
 basis_vectors:
  - 6.734, 0, 0
  - 0, 11.613, 0
  - -2.221, 0, 5.925
excitations:
 - type: DGSresolution
orientation:
 u: 1,0,0
 v: 0,1,0
shape: block width="3*mm" height="2.0*cm" thickness="3*mm"
temperature: 0.3*K

check

In [ ]:
from mcvine.workflow.sample import loadSampleYml
sample = loadSampleYml('sample.yaml')

## Configuration file

In [ ]:
%%file resolution_workflow_config.py
import os
thisdir = os.path.abspath(os.path.dirname(__file__) or '.')

import numpy as np

# instrument
from dgsres.instruments import cncs as instrument

# sample
sample_yaml = os.path.join(thisdir, 'sample.yaml')

# exp condition
beam = os.path.join(thisdir, "beam/")
Ei = 2.5004148296410698

from mcvine.workflow import singlextal as sx
psi_scan = sx.axis(min=0., max=181., step=1.)

# sim directory name
def simdir(q, E, slice):
    return 'sim-%s-q_%.3f,E_%.3f' % (slice.name, q, E)
sim_Nrounds_beam = 50

# slice
# 
class Slice_H00:
    name = 'H00'
    hkl_projection = np.array([1.,0,0])
    hkl0 = np.array([0.,0,0])
    
    class grid:
        "simulations will be done for points on this grid"
        qaxis = sx.axis(min=-0.5, max=4.1, step=0.5)
        Eaxis = sx.axis(min=-0.5, max=2.51, step=0.5)
GammaA = Slice_H00

# 
class Slice_H10:
    name = 'H10'
    hkl_projection = np.array([1.,0,0])
    hkl0 = np.array([0,1.,0])
    
    class grid:
        "simulations will be done for points on this grid"
        qaxis = sx.axis(min=-0.5, max=4.1, step=0.5)
        Eaxis = sx.axis(min=-0.5, max=2.51, step=0.5)

# 
class Slice_H20:
    name = 'H20'
    hkl_projection = np.array([1.,0,0])
    hkl0 = np.array([0,2.,0])
    
    class grid:
        "simulations will be done for points on this grid"
        qaxis = sx.axis(min=-0.5, max=4.1, step=0.5)
        Eaxis = sx.axis(min=-0.5, max=2.51, step=0.5)

# 
class Slice_0K0:
    name = '0K0'
    hkl_projection = np.array([0,1.,0])
    hkl0 = np.array([0,0,0])
    
    class grid:
        "simulations will be done for points on this grid"
        qaxis = sx.axis(min=-3, max=1, step=0.5)
        Eaxis = sx.axis(min=-0.5, max=2.51, step=0.5)

# 
class Slice_1K0:
    name = '1K0'
    hkl_projection = np.array([0,1.,0])
    hkl0 = np.array([1.,0,0])
    
    class grid:
        "simulations will be done for points on this grid"
        qaxis = sx.axis(min=-0.5, max=4.1, step=0.5)
        Eaxis = sx.axis(min=-0.5, max=2.51, step=0.5)

# 
class Slice_HH0:
    name = 'HH0'
    hkl_projection = np.array([1.,1.,0])
    hkl0 = np.array([0,0,0.])
    
    class grid:
        "simulations will be done for points on this grid"
        qaxis = sx.axis(min=-0.5, max=4.1, step=0.5)
        Eaxis = sx.axis(min=-0.5, max=2.51, step=0.5)

# 
class Slice_0p5K0:
    name = '0p5K0'
    hkl_projection = np.array([0,1.,0])
    hkl0 = np.array([0.5,0,0])
    
    class grid:
        "simulations will be done for points on this grid"
        qaxis = sx.axis(min=-0.5, max=4.1, step=0.5)
        Eaxis = sx.axis(min=-0.5, max=2.51, step=0.5)
        
#
# slices = [Slice_H00, Slice_H10, Slice_H20, Slice_0K0, Slice_1K0, Slice_HH0, Slice_0p5K0]
# slices = [Slice_H03]
slices = [Slice_0K0]

class res_2d_grid:
    "resolution data will be histogrammed into this grid"
    qaxis = sx.axis(min=-0.25, max=0.25, step=0.01)
    Eaxis = sx.axis(min=-0.5, max=0.51, step=0.02)

class fitting:
    rounds = 3
    gaussian2d_threshold = 0.5
    alpha_bounds = (-np.pi/2, np.pi/2)

for sl in slices:
    sl.res_2d_grid = res_2d_grid
    sl.fitting = fitting
    

check

In [ ]:
import imp
config = imp.load_source('config', './resolution_workflow_config.py')